In [30]:
import re
import requests
from bs4 import BeautifulSoup, NavigableString, Comment


In [31]:
url = 'https://thuvienphapluat.vn/van-ban/Dau-tu/Nghi-dinh-214-2025-ND-CP-huong-dan-Luat-Dau-thau-ve-lua-chon-nha-thau-668157.aspx'
response = requests.get(url)
response

<Response [403]>

In [32]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
response = requests.get(url, headers=headers)
print(f"Status code: {response.status_code}")

if response.status_code == 200:
    print("Truy cập thành công!")
else:
    print(f"HTTP {response.status_code} - Vẫn bị từ chối truy cập")

Status code: 200
Truy cập thành công!


In [37]:
soup = BeautifulSoup(response.content, 'html.parser')
# Lấy dữ liệu từ thẻ div class='content1'
content_div = soup.find('div', class_='content1')
content_div

<div class="content1">
<div>
<div>
<table border="0" cellpadding="0" cellspacing="0" style="width:100.0%;border-collapse:collapse" width="100%"><!--VABWAFAATABfADIAMAAyADUAMAA5ADEAMgA=-->
<tr>
<td style="width:167.4pt;padding:0cm 5.4pt 0cm 5.4pt" valign="top" width="223">
<p align="center" style="margin-top:6.0pt;text-align:center"><b><span lang="EN">CHÍNH PH</span><span lang="VI">Ủ</span></b><b><span lang="VI"><br/>
  -------</span></b></p>
</td>
<td style="width:275.4pt;padding:0cm 5.4pt 0cm 5.4pt" valign="top" width="367">
<p align="center" style="margin-top:6.0pt;text-align:center"><b><span lang="VI">CỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM<br/>
  Độc lập - Tự do - Hạnh phúc <br/>
  ---------------</span></b></p>
</td>
</tr>
<tr>
<td style="width:167.4pt;padding:0cm 5.4pt 0cm 5.4pt" valign="top" width="223">
<p align="center" style="margin-top:6.0pt;text-align:center"><span lang="VI">Số: 214/2025/NĐ-CP</span></p>
</td>
<td style="width:275.4pt;padding:0cm 5.4pt 0cm 5.4pt" valign="top" wi

In [38]:
def extract_content_from_div(content_div):
    """
    Trích xuất nội dung từ div và chuyển đổi sang markdown
    """
    if not content_div:
        return "Không tìm thấy nội dung"
    
    markdown_content = []
    
    # Duyệt qua tất cả các thẻ con
    for element in content_div.find_all(recursive=True):
        # Bỏ qua các comment và navigable string
        if isinstance(element, (Comment, NavigableString)):
            continue
            
        # Xử lý các thẻ tiêu đề
        if element.name in ['h1', 'h2', 'h3', 'h4', 'h5', 'h6']:
            level = int(element.name[1])
            text = element.get_text(strip=True)
            if text:
                markdown_content.append('#' * level + ' ' + text + '\n')
        
        # Xử lý thẻ paragraph
        elif element.name == 'p':
            text = element.get_text(strip=True)
            if text:
                markdown_content.append(text + '\n\n')
        
        # Xử lý danh sách có thứ tự
        elif element.name == 'ol':
            for i, li in enumerate(element.find_all('li'), 1):
                text = li.get_text(strip=True)
                if text:
                    markdown_content.append(f"{i}. {text}\n")
            markdown_content.append('\n')
        
        # Xử lý danh sách không thứ tự
        elif element.name == 'ul':
            for li in element.find_all('li'):
                text = li.get_text(strip=True)
                if text:
                    markdown_content.append(f"- {text}\n")
            markdown_content.append('\n')
        
        # Xử lý các thẻ strong/b
        elif element.name in ['strong', 'b']:
            text = element.get_text(strip=True)
            if text:
                markdown_content.append(f"**{text}**")
        
        # Xử lý các thẻ em/i
        elif element.name in ['em', 'i']:
            text = element.get_text(strip=True)
            if text:
                markdown_content.append(f"*{text}*")
        
        # Xử lý thẻ div với class đặc biệt
        elif element.name == 'div':
            text = element.get_text(strip=True)
            # Chỉ thêm nội dung nếu div không có thẻ con phức tạp
            if text and not element.find_all(['div', 'p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6']):
                markdown_content.append(text + '\n\n')
    
    return ''.join(markdown_content)

# Test function
if content_div:
    extracted_content = extract_content_from_div(content_div)
    print("Nội dung đã trích xuất (100 ký tự đầu):")
    print(extracted_content[:500] + "..." if len(extracted_content) > 500 else extracted_content)

Nội dung đã trích xuất (100 ký tự đầu):
CHÍNH PHỦ-------

**CHÍNH PHỦ****-------**CỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAMĐộc lập - Tự do - Hạnh phúc---------------

**CỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAMĐộc lập - Tự do - Hạnh phúc---------------**Số: 214/2025/NĐ-CP

Hà
  Nội, ngày 04 tháng 8 năm 2025

*Hà
  Nội, ng**ày 04 tháng 8 năm 2025*NGHỊ
ĐỊNH

**NGHỊ
ĐỊNH**QUY ĐỊNH CHI TIẾT MỘT
SỐ ĐIỀU VÀ BIỆN PHÁP THI HÀNH LUẬT ĐẤU THẦU VỀ LỰA CHỌN NHÀ THẦU

Căn cứLuật Tổ chức Chính phủ số 63/2025/QH15;

*Căn cứLuật Tổ chức Chính phủ số 63/2025/Q...


In [39]:
def extract_clean_content(content_div):
    """
    Trích xuất nội dung sạch và chuyển đổi sang markdown
    """
    if not content_div:
        return "Không tìm thấy nội dung"
    
    markdown_lines = []
    processed_elements = set()
    
    def clean_text(text):
        """Làm sạch text, loại bỏ khoảng trắng thừa"""
        return ' '.join(text.split())
    
    def process_element(element, level=0):
        """Xử lý từng element một cách đệ quy"""
        if element in processed_elements:
            return
        
        processed_elements.add(element)
        
        # Xử lý tiêu đề
        if element.name in ['h1', 'h2', 'h3', 'h4', 'h5', 'h6']:
            level_num = int(element.name[1])
            text = clean_text(element.get_text())
            if text and len(text) > 2:  # Chỉ lấy tiêu đề có ý nghĩa
                markdown_lines.append('#' * level_num + ' ' + text + '\n\n')
        
        # Xử lý paragraph
        elif element.name == 'p':
            text = clean_text(element.get_text())
            if text and len(text) > 10:  # Chỉ lấy đoạn văn có ý nghĩa
                markdown_lines.append(text + '\n\n')
        
        # Xử lý div có nội dung văn bản
        elif element.name == 'div':
            # Nếu div không chứa các thẻ block khác, lấy text
            if not element.find_all(['div', 'p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'ul', 'ol']):
                text = clean_text(element.get_text())
                if text and len(text) > 10:
                    markdown_lines.append(text + '\n\n')
            else:
                # Nếu có thẻ con, xử lý đệ quy
                for child in element.children:
                    if hasattr(child, 'name') and child.name:
                        process_element(child, level + 1)
        
        # Xử lý danh sách
        elif element.name == 'ul':
            for li in element.find_all('li'):
                text = clean_text(li.get_text())
                if text:
                    markdown_lines.append(f"- {text}\n")
            markdown_lines.append('\n')
        
        elif element.name == 'ol':
            for i, li in enumerate(element.find_all('li'), 1):
                text = clean_text(li.get_text())
                if text:
                    markdown_lines.append(f"{i}. {text}\n")
            markdown_lines.append('\n')
    
    # Bắt đầu xử lý từ root element
    process_element(content_div)
    
    # Làm sạch kết quả
    content = ''.join(markdown_lines)
    # Loại bỏ nhiều dòng trống liên tiếp
    content = re.sub(r'\n{3,}', '\n\n', content)
    
    return content.strip()

# Test improved function
if content_div:
    clean_content = extract_clean_content(content_div)
    print(f"Độ dài nội dung: {len(clean_content)} ký tự")
    print("\nNội dung đã trích xuất (1000 ký tự đầu):")
    print(clean_content[:1000] + "..." if len(clean_content) > 1000 else clean_content)

Độ dài nội dung: 423621 ký tự

Nội dung đã trích xuất (1000 ký tự đầu):
QUY ĐỊNH CHI TIẾT MỘT SỐ ĐIỀU VÀ BIỆN PHÁP THI HÀNH LUẬT ĐẤU THẦU VỀ LỰA CHỌN NHÀ THẦU

Căn cứ Luật Tổ chức Chính phủ số 63/2025/QH15;

Căn cứ Luật Tổ chức chính quyền địa phương số 72/2025/QH15;

Căn cứ Luật Đấu thầu số 22/2023/QH15 được sửa đổi, bổ sung bởi Luật số 57/2024/QH15, Luật số 90/2025/QH15;

Theo đề nghị của Bộ trưởng Bộ Tài chính;

Chính phủ ban hành Nghị định quy định chi tiết một số điều và biện pháp thi hành Luật Đấu thầu về lựa chọn nhà thầu.

NHỮNG QUY ĐỊNH CHUNG

Điều 1. Phạm vi điều chỉnh

1. Nghị định này quy định chi tiết một số điều của Luật Đấu thầu về lựa chọn nhà thầu, bao gồm: khoản 5 Điều 3; khoản 1 Điều 5; khoản 6 Điều 6; khoản 6 Điều 10; khoản 3 Điều 15; khoản 4 Điều 19; khoản 2 Điều 20; Điều 23; khoản 1 Điều 24; khoản 2 Điều 29; khoản 2 Điều 29a; khoản 3 Điều 29b; khoản 4 Điều 36; khoản 2 Điều 39; khoản 2 Điều 43; khoản 2 và khoản 4 Điều 44; khoản 3 Điều 45; Điều 50; khoản 3 và khoản 

In [40]:
import os
from datetime import datetime
from urllib.parse import urlparse

def export_to_markdown(content, url, output_dir="../processed/"):
    """
    Export nội dung ra file markdown
    """
    # Tạo thư mục nếu chưa có
    os.makedirs(output_dir, exist_ok=True)
    
    # Tạo tên file từ URL
    parsed_url = urlparse(url)
    path_parts = parsed_url.path.split('/')
    
    # Lấy phần cuối của URL làm tên file, loại bỏ extension
    filename_base = path_parts[-1].replace('.aspx', '').replace('.html', '')
    if not filename_base:
        filename_base = "crawled_content"
    
    # Tạo timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{filename_base}_{timestamp}.md"
    filepath = os.path.join(output_dir, filename)
    
    # Tạo header cho file markdown
    header = f"""---
title: "Nội dung từ {parsed_url.netloc}"
url: {url}
crawled_at: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
source: thuvienphapluat.vn
---

"""
    
    # Ghi file
    try:
        with open(filepath, 'w', encoding='utf-8') as f:
            f.write(header)
            f.write(content)
        
        print(f"✅ Đã export thành công vào file: {filepath}")
        print(f"📄 Kích thước file: {os.path.getsize(filepath):,} bytes")
        return filepath
    
    except Exception as e:
        print(f"❌ Lỗi khi export file: {str(e)}")
        return None

# Export nội dung đã trích xuất
if 'clean_content' in locals() and clean_content:
    output_file = export_to_markdown(clean_content, url)
    
    if output_file:
        # Hiển thị thông tin về file đã tạo
        print(f"\n📊 Thông tin file đã tạo:")
        print(f"- Đường dẫn: {output_file}")
        print(f"- Số dòng: {len(clean_content.splitlines())}")
        print(f"- Số từ (ước tính): {len(clean_content.split())}")
else:
    print("❌ Chưa có nội dung để export")

✅ Đã export thành công vào file: ../processed/Nghi-dinh-214-2025-ND-CP-huong-dan-Luat-Dau-thau-ve-lua-chon-nha-thau-668157_20250929_120921.md
📄 Kích thước file: 573,613 bytes

📊 Thông tin file đã tạo:
- Đường dẫn: ../processed/Nghi-dinh-214-2025-ND-CP-huong-dan-Luat-Dau-thau-ve-lua-chon-nha-thau-668157_20250929_120921.md
- Số dòng: 4149
- Số từ (ước tính): 94805


In [41]:
def crawl_and_export_to_markdown(url, output_dir="../processed/"):
    """
    Function hoàn chỉnh để crawl và export sang markdown
    """
    print(f"🔄 Bắt đầu crawl: {url}")
    
    # Headers để tránh bị block
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    try:
        # 1. Gửi request
        response = requests.get(url, headers=headers, timeout=30)
        print(f"📡 Status code: {response.status_code}")
        
        if response.status_code != 200:
            print(f"❌ Không thể truy cập trang web. Status: {response.status_code}")
            return None
        
        # 2. Parse HTML
        soup = BeautifulSoup(response.content, 'html.parser')
        content_div = soup.find('div', class_='content1')
        
        if not content_div:
            print("❌ Không tìm thấy div với class 'content1'")
            return None
        
        print("✅ Tìm thấy nội dung")
        
        # 3. Trích xuất nội dung
        clean_content = extract_clean_content(content_div)
        print(f"📝 Đã trích xuất {len(clean_content)} ký tự")
        
        # 4. Export ra markdown
        output_file = export_to_markdown(clean_content, url, output_dir)
        
        return output_file
        
    except requests.RequestException as e:
        print(f"❌ Lỗi khi request: {str(e)}")
        return None
    except Exception as e:
        print(f"❌ Lỗi không xác định: {str(e)}")
        return None

# Test function với URL hiện tại
print("=== TEST CRAWL AND EXPORT FUNCTION ===")
result = crawl_and_export_to_markdown(url)

if result:
    print(f"\n🎉 Hoàn thành! File đã được lưu tại: {result}")
else:
    print("\n❌ Crawl không thành công")

=== TEST CRAWL AND EXPORT FUNCTION ===
🔄 Bắt đầu crawl: https://thuvienphapluat.vn/van-ban/Dau-tu/Nghi-dinh-214-2025-ND-CP-huong-dan-Luat-Dau-thau-ve-lua-chon-nha-thau-668157.aspx
📡 Status code: 200
✅ Tìm thấy nội dung
📝 Đã trích xuất 423621 ký tự
✅ Đã export thành công vào file: ../processed/Nghi-dinh-214-2025-ND-CP-huong-dan-Luat-Dau-thau-ve-lua-chon-nha-thau-668157_20250929_120957.md
📄 Kích thước file: 573,613 bytes

🎉 Hoàn thành! File đã được lưu tại: ../processed/Nghi-dinh-214-2025-ND-CP-huong-dan-Luat-Dau-thau-ve-lua-chon-nha-thau-668157_20250929_120957.md


In [ ]:
def batch_crawl_urls(urls, output_dir="../processed/", delay=2):
    """
    Crawl hàng loạt nhiều URL
    """
    import time
    
    results = []
    total_urls = len(urls)
    
    print(f"🚀 Bắt đầu crawl {total_urls} URL...")
    print(f"⏱️  Delay giữa các request: {delay} giây")
    print("=" * 50)
    
    for i, url in enumerate(urls, 1):
        print(f"\n[{i}/{total_urls}] Processing: {url}")
        
        result = crawl_and_export_to_markdown(url, output_dir)
        results.append({
            'url': url,
            'success': result is not None,
            'output_file': result,
            'index': i
        })
        
        # Delay giữa các request để tránh bị block
        if i < total_urls:
            print(f"⏳ Chờ {delay} giây...")
            time.sleep(delay)
    
    # Tóm tắt kết quả
    print("\n" + "=" * 50)
    print("📊 TỔNG KẾT:")
    successful = sum(1 for r in results if r['success'])
    failed = total_urls - successful
    
    print(f"✅ Thành công: {successful}/{total_urls}")
    print(f"❌ Thất bại: {failed}/{total_urls}")
    
    if successful > 0:
        print(f"\n📁 Các file đã tạo:")
        for result in results:
            if result['success']:
                print(f"  - {result['output_file']}")
    
    return results

# Example: Danh sách URL để test (có thể thêm nhiều URL khác)
sample_urls = [
    'https://thuvienphapluat.vn/van-ban/Dau-tu/Nghi-dinh-214-2025-ND-CP-huong-dan-Luat-Dau-thau-ve-lua-chon-nha-thau-668157.aspx',
    # Có thể thêm URL khác ở đây
    # 'https://thuvienphapluat.vn/van-ban/...'
]

print("📝 Sample URLs để crawl:")
for i, url in enumerate(sample_urls, 1):
    print(f"{i}. {url}")

# Uncomment dòng dưới để chạy batch crawl
# batch_results = batch_crawl_urls(sample_urls[:1])  # Chỉ test với 1 URL đầu tiên

# 📖 Hướng dẫn sử dụng Web Crawler và Export Markdown

## 🎯 Tính năng chính

1. **Crawl nội dung từ thuvienphapluat.vn**
2. **Trích xuất nội dung sạch** từ div có class 'content1'
3. **Chuyển đổi sang định dạng Markdown** với header, paragraphs, lists
4. **Export ra file .md** với metadata và timestamp
5. **Hỗ trợ crawl hàng loạt** nhiều URL

## 🚀 Cách sử dụng

### 1. Crawl một URL đơn lẻ:
```python
url = 'https://thuvienphapluat.vn/van-ban/...'
output_file = crawl_and_export_to_markdown(url)
```

### 2. Crawl nhiều URL:
```python
urls = [
    'https://thuvienphapluat.vn/van-ban/url1',
    'https://thuvienphapluat.vn/van-ban/url2',
    # ...
]
results = batch_crawl_urls(urls, delay=3)  # delay 3 giây giữa các request
```

### 3. Tùy chỉnh thư mục output:
```python
crawl_and_export_to_markdown(url, output_dir="./my_crawled_data/")
```

## 📁 Cấu trúc file output

File markdown được tạo sẽ có:
- **Header metadata** (title, url, timestamp)
- **Nội dung đã được làm sạch** và format markdown
- **Tên file** theo format: `{original_name}_{timestamp}.md`

## ⚠️ Lưu ý quan trọng

1. **Respect robots.txt** - Luôn kiểm tra và tuân thủ robots.txt của website
2. **Delay giữa requests** - Sử dụng delay hợp lý (2-3 giây) để tránh bị block
3. **Error handling** - Function đã có xử lý lỗi cơ bản
4. **Content validation** - Kiểm tra content trước khi sử dụng

## 🔧 Customization

Bạn có thể tùy chỉnh:
- **CSS selectors** trong `extract_clean_content()`
- **Markdown formatting** rules
- **File naming** convention
- **Output directory** structure